In [1]:
import logging
import requests 
import datetime
import time
import pandas as pd
import yfinance as yf 
from bs4 import BeautifulSoup
import requests
import re 
import constants

stock = 'GOOGL'
PATH = 'stocks/tickers'

# lookup stock in file 
file = f'{PATH}/{stock}.csv'
news = pd.read_csv(file, parse_dates=['Date'])

# calculate intraday return in yfinance
def get_intraday_return(stock, period = '1mo'): 
    data = yf.download(stock, period = '1mo', interval = '1d')  
    data.columns = data.columns.droplevel(1)                # drop the multi-level column index
    data.columns.name = None 
    data['Prev Close'] = data['Close'].shift(1)     # get previous close by shifting it down by 1
    data['% Change'] = ((data['Close'] - data['Prev Close']) / data['Prev Close'])  * 100
    data['% Change'] = data['% Change'].round(2)
    data.sort_values('% Change', ascending = False, inplace = True)
    return data

swings = get_intraday_return('GOOGL')               # largest % price changes 

def lookup_date(timestamp):

    '''Looks up the price movement on a given date'''
    try:
        return swings.loc[str(timestamp.date()), '% Change']
    except KeyError:
        return None     # print(f'No price movement data for {timestamp.date()}')

news['% Change'] = news['Date'].apply(lookup_date)         
news

[*********************100%***********************]  1 of 1 completed


,Date,Title,Link,Source,% Change
0,2024-11-25 21:38:00,"Google, Meta urge Australia to delay bill on s...",https://finance.yahoo.com/news/google-meta-urg...,Reuters,NaN
1,2024-11-25 18:12:00,Google Arguments Draw Skepticism From Judge in...,https://finance.yahoo.com/news/google-argument...,Bloomberg,NaN
2,2024-11-25 08:00:00,AI Boom Propels a Wild-West-Era Texas Landowne...,https://finance.yahoo.com/news/ai-boom-propels...,Bloomberg,NaN
3,2024-11-25 06:04:00,Google's US antitrust trial over online ad emp...,https://finance.yahoo.com/news/googles-us-anti...,Reuters,NaN
4,2024-11-22 11:37:00,Amazon Invests an Additional $4 Billion in AI ...,https://finance.yahoo.com/news/amazon-investin...,Bloomberg,NaN
...,...,...,...,...,...
64,2024-12-20 07:00:00,Everybody Wants to Be My Friend: Trump Marvels...,https://finance.yahoo.com/news/everybody-wants...,Bloomberg,1.54
65,2024-12-20 13:00:00,OpenAI Unveils More Advanced Reasoning Model i...,https://finance.yahoo.com/news/openai-unveils-...,Bloomberg,1.54
66,2024-12-20 14:55:00,OpenAI unveils 'o3' reasoning AI models in tes...,https://finance.yahoo.com/news/openai-unveils-...,Reuters,1.54
67,2024-12-20 22:04:00,Google offers to loosen search deals in US ant...,https://finance.yahoo.com/news/google-says-cou...,Reuters,1.54


In [6]:
def get_google_results(query):      # using google Custom Search JSON API

    params = {
        'key': constants.GOOGLE_API_KEY, 
        'cx': constants.SEARCH_ENGINE_ID, 
    }

    try: 
        params['q'] = f'"{query}"'
        response = requests.get('https://www.googleapis.com/customsearch/v1', params = params).json() 
        num_results = int(response['queries']['request'][0]['totalResults'])
        print(query, num_results)
        return num_results
    
    except Exception as e:
        print("Could not load results!")
        raise e

    
'''
Mines the top N days with the highest price change for news catalysts
'''
n = 1
news.loc[date where % change is highest, 'News'] = get_google_results(stock)  # search for news catalysts   
# search for potential news catalysts by looking at biggest price change days
change_prices = news['% Change'].dropna().drop_duplicates().sort_values(ascending=False).reset_index(drop=True)
change_prices



Baillie Cools on Tech in Favor of Boring Infrastructure Stocks 68
Google invests in Intersect Power in over $800 million funding round amid AI race 10
Cloud Computing Tax Threatens Chicagos Silicon Valley Dream 1
Google parent Alphabet jumps on quantum chip breakthrough 6060
Brussels seeks more information on secret Google-Meta ads deal targeting teens, FT reports 134
Alphabet Climbs After Breakthrough With Willow Quantum Chip 7
Google's biggest bet is AI for search, investment chief says 4460
Google asks FTC to break up Microsoft's cloud deal with OpenAI, the Information says 2910
GM Calls It Quits on Mary Barras $50 Billion Robotaxi Dream 2
Googles Top Lobbyist Departs During Continued Regulatory Scrutiny 1


C:\Users\patrick4\AppData\Local\Temp\ipykernel_17492\2521001307.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_candidates['hits'] = news_candidates['Title'].apply(get_google_results)


,Date,Title,Link,Source,hits
30,2024-12-10 01:02:00,Baillie Cools on Tech in Favor of Boring Infra...,https://finance.yahoo.com/news/baillie-cools-t...,Bloomberg,68
31,2024-12-10 09:09:00,Google invests in Intersect Power in over $800...,https://finance.yahoo.com/news/google-invests-...,Reuters,10
32,2024-12-10 09:49:00,Cloud Computing Tax Threatens Chicagos Silicon...,https://finance.yahoo.com/news/cloud-computing...,Bloomberg,1
33,2024-12-10 11:31:00,Google parent Alphabet jumps on quantum chip b...,https://finance.yahoo.com/news/google-parent-a...,Reuters,6060
34,2024-12-10 12:18:00,Brussels seeks more information on secret Goog...,https://finance.yahoo.com/m/b3e11604-af45-3fc3...,Reuters,134
35,2024-12-10 16:10:00,Alphabet Climbs After Breakthrough With Willow...,https://finance.yahoo.com/news/alphabet-climbs...,Bloomberg,7
36,2024-12-10 18:19:00,"Google's biggest bet is AI for search, investm...",https://ca.finance.yahoo.com/news/googles-bigg...,Reuters,4460
37,2024-12-10 19:32:00,Google asks FTC to break up Microsoft's cloud ...,https://finance.yahoo.com/news/google-asks-ftc...,Reuters,2910
38,2024-12-10 19:49:00,GM Calls It Quits on Mary Barras $50 Billion R...,https://finance.yahoo.com/news/gm-calls-quits-...,Bloomberg,2
39,2024-12-10 20:26:00,Googles Top Lobbyist Departs During Continued ...,https://finance.yahoo.com/news/google-top-lobb...,Bloomberg,1


In [58]:
# write back enriched news data back to csv 
# check if data is corrupted, if so do not rewrite to file.



0     5.59
1     5.52
2     3.60
3     1.77
4     1.75
5     1.54
6     1.20
7     0.88
8     0.49
9     0.07
10   -0.09
11   -0.99
12   -1.11
13   -1.71
14   -1.76
15   -3.59
Name: % Change, dtype: float64